### Configure Azure ML Credentials & Compute Target

In [35]:
#import required libraries
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

#Enter details of your Azure Machine Learning workspace
subscription_id = "d69de6af-cb80-43c4-9527-d8d7eb3180f5"
resource_group = "lesterjt-rg"
workspace = "Sydney"

#connect to the workspace
ml_client = MLClient(DefaultAzureCredential(), subscription_id, resource_group, workspace)

In [37]:
from azure.ai.ml.entities import AmlCompute

# specify aml compute name.
cpu_compute_target = "sydney-compute-cpu"

try:
    ml_client.compute.get(cpu_compute_target)
except Exception:
    print("Creating a new cpu compute target...")
    compute = AmlCompute(
        name=cpu_compute_target, size="STANDARD_D2_V2", min_instances=0, max_instances=4
    )
    ml_client.compute.begin_create_or_update(compute).result()

### Submit the Training Job to Azure ML Compute Resource

In [ ]:
from azure.ai.ml import command

# define the command
command_job = command(
    code="./",
    command="python train.py",
    environment="AzureML-acpt-pytorch-2.2-cuda12.1@latest",
    compute=cpu_compute_target,
    experiment_name="train-on-cpu",
    display_name="train-image-classifier"
)

# submit the command
returned_job = ml_client.jobs.create_or_update(command_job)
# get a URL for the status of the job
returned_job.studio_url

### Register Model & Save Locally

In [ ]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

# Get the completed job
job_details = ml_client.jobs.get(returned_job.name)

# Register the model from the job's outputs
registered_model = ml_client.models.create_or_update(
    Model(
        path=f"{job_details.outputs['artifacts']}/outputs/elephant_classifier_resnet50.pth",  # Job output path
        name="image-classifier",
        description="Trained image classifier",
    )
)

ml_client.models.download(registered_model, "elephant_classifier_resnet50.pth")
